In [1]:
from igramscraper.instagram import Instagram
import pandas as pd
import time
import random
from sqlalchemy import create_engine

ModuleNotFoundError: No module named 'sqlalchemy'

In [4]:
#connect to the instagram scraper
instagram = Instagram()

In [6]:
# i didn't use all of the function at the end, because it didn't work out with the limits/ difficulties with instagram, but in theory all of this methods should work smoothly

#getting the number of posts on instagram and get the account id
def get_post_count(username):
    account = instagram.get_account(username)
    posts_count = account.media_count
    return posts_count,account.identifier

#out of the username, get all the medias and form them to a df (i called the method df_format() in this method)
def create_df(username):
    print("start")
    time.sleep(random.randint(30,60))
    number_posts,account_id = get_post_count(username)
    print("Number of posts")
    time.sleep(random.randint(30,60))
    medias = instagram.get_medias(username, number_posts)
    df = df_format(medias,username)
    return df

#forms a dataframe out of the data
def df_format(medias,username,account_id):
    list_ids = []
    no_likes = []
    links=[]
    caption =[]
    for i in range(len(medias)):
        list_ids.append(medias[i].identifier)
        no_likes.append(medias[i].likes_count)
        links.append(medias[i].link)
        caption.append(medias[i].caption)
    df = pd.DataFrame(list(zip(list_ids,caption,no_likes,links)),columns=['post_id','caption','likes','link'])
    df['username'] = username
    df['account_id'] = account_id
    return df

# parsing a df into the database
def to_sqlbase(df):
    con = create_engine('mysql+pymysql://************@*************/examination_sma')
    conn = con.connect()
    for i in range(len(df)):
        if df.at[i,'caption'] != None:
            caption_text = df.at[i,'caption'].replace('"','""').replace('%','%%').replace("'","''")
        conn.execute("INSERT INTO instagram (post_id,caption,likes,link,username,account_id) VALUES ('{0}','{1}',{2},'{3}','{4}','{5}')".format(str(df.at[i,'post_id']),str(caption_text),df.at[i,'likes'],str(df.at[i,'link']),str(df.at[i,'username']),str(df.at[i,'account_id'])))

In [201]:
# get number of post and account id of daviddobrik
number_posts,account_id = get_post_count("daviddobrik")

In [202]:
#get medias of daviddobrik
medias = instagram.get_medias("daviddobrik", 466)

In [218]:
#form medias of daviddobrik to df 
df_david = df_format(medias,'daviddobrik','608037384')

In [269]:
#parse david_df into table in database
to_sqlbase(df_david)

In [233]:
#get number of post and account id of joebiden
number_posts,account_id = get_post_count("joebiden")

In [234]:
#shrink to a maximum of 466, because at a count over 500 the scraper just stops and didnt send anything back
number_posts

1316

In [253]:
#get medias of joebiden
medias = Instagram().get_medias("joebiden",466)

In [254]:
#form medias of joebiden to df 
df_biden = df_format(medias,'joebiden','5511100943')

In [270]:
#parse df_biden into table in database
to_sqlbase(df_biden)

In [7]:
#get number of post and account id of gretathunberg
number_posts,account_id = get_post_count("gretathunberg")

In [8]:
number_posts

729

In [261]:
#get medias of gretathunberg
medias = instagram.get_medias("gretathunberg",466)

In [263]:
#form medias of gretathunbergto df 
df_thunberg = df_format(medias,'gretathunberg','4044526946')

In [271]:
#parse df_thunberg into table in database
to_sqlbase(df_thunberg)